In [3]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [4]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x = vectorize_sequences(train_data)
# Our vectorized test data
x_val = vectorize_sequences(test_data)

In [5]:
# Our vectorized labels
y = np.asarray(train_labels).astype('float32')
y_val = np.asarray(test_labels).astype('float32')

In [6]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
import keras

# callback is sent to the model through callbacks parameter of fit call
# this parameter receives callback list
# we can specify as many as parameters we want
callbacks_list = [
    # stop train on obtaining no more improvement
    keras.callbacks.EarlyStopping(
        monitor='val_acc', # monitor model's accuracy to validation data
#        patience=1,        # if no improvement among 2 epochs,
        patience=2,        # if no improvement among 3 epochs,
                           # stop train
    ),
    # save the current weighs on each epoch
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5', # file path to save model
        monitor='val_loss',     # these two paramters mean that
        save_best_only=True     # we do not overwrite model file 
                                # without improvement 
    )
]

# in this case we monitor accuracy
# then accuracy must be part of metrics of model
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

# this callback monitors val_loss and val_acc,
# so we need to specify valildation_data for fit call
model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 245us/step - loss: 0.1604 - acc: 0.9420 - val_loss: 0.3408 - val_acc: 0.8773
Epoch 2/10
25000/25000 [==============================] - 6s 235us/step - loss: 0.1419 - acc: 0.9506 - val_loss: 0.3651 - val_acc: 0.8766
Epoch 3/10
25000/25000 [==============================] - 6s 228us/step - loss: 0.1264 - acc: 0.9571 - val_loss: 0.3893 - val_acc: 0.8736


In [16]:
import keras

# callback is sent to the model through callbacks parameter of fit call
# this parameter receives callback list
# we can specify as many as parameters we want
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', # monitor loss value for validation data
        factor=0.1,         # divide learning rate by 10 on callback activation
        patience=10,        # if no improvement among 10 epochs,
                            # activate callback
    )
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

# this callback monitors val_loss,
# so we need to specify valildation_data for fit call
model.fit(x, y,
         epochs=20,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 245us/step - loss: 0.0151 - acc: 0.9963 - val_loss: 1.2785 - val_acc: 0.8513
Epoch 2/20
25000/25000 [==============================] - 6s 247us/step - loss: 0.0133 - acc: 0.9972 - val_loss: 1.4030 - val_acc: 0.8519
Epoch 3/20
25000/25000 [==============================] - 6s 243us/step - loss: 0.0108 - acc: 0.9977 - val_loss: 1.6736 - val_acc: 0.8512
Epoch 4/20
25000/25000 [==============================] - 6s 249us/step - loss: 0.0099 - acc: 0.9979 - val_loss: 1.6562 - val_acc: 0.8498
Epoch 5/20
25000/25000 [==============================] - 6s 232us/step - loss: 0.0082 - acc: 0.9985 - val_loss: 1.9209 - val_acc: 0.8509
Epoch 6/20
25000/25000 [==============================] - 6s 241us/step - loss: 0.0079 - acc: 0.9986 - val_loss: 1.9481 - val_acc: 0.8500
Epoch 7/20
25000/25000 [==============================] - 6s 237us/step - loss: 0.0063 - acc: 0.9990 - val_loss: 2.1867 - 

In [26]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        # called from perent model before training,
        # inform calling model to callback
        self.model = model
        
        layer_outputs = [layer.output for layer in model.layers]
        
        # model instance to return each layer's activation
        self.activations_model = keras.models.Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        print('on_epoch_end()')
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        # get the first sample of validation data
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)

        # save activations array to disc
        f = open('activaions_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        #np.save(f, activations)
        f.close()

In [27]:
import keras

# callback is sent to the model through callbacks parameter of fit call
# this parameter receives callback list
# we can specify as many as parameters we want
callbacks_list = [
    ActivationLogger()
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

# custom callback ActivaionLogger needs valildation_data for fit call
model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 248us/step - loss: 0.0030 - acc: 0.9995 - val_loss: 2.8751 - val_acc: 0.8460
on_epoch_end()


ValueError: could not broadcast input array from shape (16) into shape (1)